In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import OneHotEncoder 

In [2]:
df = pd.read_csv("datatrain.csv")
df

,Suhu,Waktu,Kondisi Langit,Kelembapan,Terbang/Tidak
0,Normal,Siang,Berawan,Normal,Tidak
1,Normal,Siang,Cerah,Normal,Tidak
2,Normal,Pagi,Hujan,Normal,Tidak
3,Normal,Sore,Rintik,Rendah,Ya
4,Normal,Malam,Berawan,Tinggi,Tidak
...,...,...,...,...,...
75,Tinggi,Pagi,Hujan,Rendah,Tidak
76,Tinggi,Pagi,Berawan,Rendah,Ya
77,Tinggi,Malam,Berawan,Normal,Tidak
78,Tinggi,Malam,Rintik,Rendah,Ya


In [3]:
enc = OneHotEncoder(sparse = False)
data_train = enc.fit_transform(df).astype(int)
data_train = np.delete(data_train,14,1) #ujiValiditas
data_train

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 1, 0, 1],
       [0, 0, 1, ..., 0, 1, 0]])

In [4]:
dt = pd.read_csv("datatest.csv")
dt

,Suhu,Waktu,Kondisi Langit,Kelembapan,Terbang/Tidak
0,Tinggi,Siang,Berawan,Rendah,NaN
1,Rendah,Siang,Hujan,Tinggi,NaN
2,Normal,Pagi,Hujan,Tinggi,NaN
3,Tinggi,Siang,Hujan,Normal,NaN
4,Rendah,Malam,Hujan,Rendah,NaN
5,Normal,Sore,Cerah,Normal,NaN
6,Tinggi,Siang,Rintik,Rendah,NaN
7,Rendah,Siang,Cerah,Rendah,NaN
8,Normal,Siang,Cerah,Rendah,NaN
9,Tinggi,Siang,Hujan,Rendah,NaN


In [5]:
enc = OneHotEncoder(sparse = False)
data_uji = enc.fit_transform(df).astype(int)
data_uji = np.delete(data_uji,15,1) #ujiValiditas
data_uji = np.delete(data_uji,14,1)
data_uji

array([[1, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 1, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 1, 0],
       [0, 0, 1, ..., 0, 0, 1]])

In [6]:
def populasi(pop):
    population = []
    for i in range (pop) :
        gen = random.choices([0,1], k=30)
        population.append(np.array(gen))
    return population

In [7]:
def cekAja(a,b):
    return np.any(np.logical_and(a,b))

In [8]:
def cekRule(kromosom, data):
#     print("222",kromosom)
    splitKrom = np.split(kromosom, [3, 7, 11, 14])
    splitData = np.split(data, [3, 7, 11, 14])

    suhu = cekAja(splitKrom[0], splitData[0])
    waktu = cekAja(splitKrom[1], splitData[1])
    langit = cekAja(splitKrom[2], splitData[2])
    lembab = cekAja(splitKrom[3], splitData[3])
    
    hasil = np.all([suhu, waktu, langit, lembab])
    if (hasil) :
        if(kromosom[14] == data[14]):
            return True
        else:
            return False
    else:
        if(kromosom[14] != data[14]):
            return True
        else:
            return False

In [9]:
def panjangKrom(kromosom, data_train):
#     print(len(kromosom))
    rules = np.split(kromosom, len(kromosom)//15)
    
    for rule in rules:
        if(cekRule(rule, data_train)):
            return True
    return False

In [10]:
def hitungFitness(dataSet, kromosom):
    benar = 0
    for data in dataSet:
        if(panjangKrom(kromosom, data)):
            benar += 1
    return benar/len(dataSet)*100

In [11]:
def crossover(p1, p2):
    titik1, titik2 = random.sample(range(len(p1)), 2)
    if titik1 > titik2:
        titik1,titik2 = titik2,titik1
    
    gen = titik2 - titik1
    gap = gen%15
    kemungkinan = [[titik1, titik1+gap], [titik1, titik1+gen], [titik2-gap, titik2]]
    titik3 = random.randrange(0, len(p2))
    
    if titik3 - gap < 0:
        titik4 = titik3+gap
    else:
        titik4 = titik3-gap
        
    if titik3 > titik4:
        titik3,titik4 = titik4,titik3
    
    idx = random.choice(kemungkinan)
    potonganP1 = np.split(p1, idx)
    potonganP2 = np.split(p2, [titik3, titik4])
    
    child1 = np.hstack([potonganP1[0], potonganP2[1], potonganP1[2]])
    child2 = np.hstack([potonganP2[0], potonganP1[1], potonganP2[2]])
    return child1, child2

In [12]:
def rouletteWheel(populasi):
    maks = 0
    for i in range (len(populasi)):
        maks = maks+populasi[i][0]
    pick = random.uniform(0, maks)
    current = 0
    for data in populasi:
        current += data[0]
        if current > pick:
            return np.array(data[1])

In [13]:
def mutasi(child1, child2):
    mungkinMutasi = random.uniform(0, 1)
    for c1 in child1:
        if mungkinMutasi < 0.2:
            if c1 == 1:
                c1 = 0
            else:
                c1 = 1
    mungkinMutasi = random.uniform(0, 1)
    for c2 in child2:
        if mungkinMutasi < 0.2:
            if c2 == 1:
                c2 = 0
            else:
                c2 = 1
    return child1, child2

# main program

In [14]:
nPop = 20 
pop = populasi(nPop)
fitKrom = []
for i in range(nPop):
    fitKrom.append([hitungFitness(data_train, pop[i]), pop[i].tolist()])
fitKrom.sort(reverse=True) #besar ke kecil

while fitKrom[0][0] != 100:
    newGen = []
    for i in range(len(fitKrom)//2):
        p1 = np.array(rouletteWheel(fitKrom)).copy()
        p2 = np.array(rouletteWheel(fitKrom)).copy()
        child1, child2 = crossover(p1, p2)
        child1, child2 = mutasi(child1, child2)
        newGen.append([hitungFitness(data_train, child1), child1.tolist()])
        newGen.append([hitungFitness(data_train, child2), child2.tolist()])
    newGen.sort()
    fitKrom[nPop//2:] = newGen[:nPop//2]
    fitKrom.sort(reverse = True)
print("HASIL AKHIR")
print(*fitKrom, sep="\n")

HASIL AKHIR
[100.0, [1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1]]
[100.0, [1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0]]
[100.0, [1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]]
[98.75, [1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1]]
[97.5, [0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1]]
[92.5, [0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0]]
[91.25, [1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1]]
[91.25, [1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0]]
[87.5, [1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1]]
[66.25, [1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1

In [15]:
def cekFinal(kromosom, data):
    splitKrom = np.split(kromosom, [3, 7, 11, 14])
    splitData = np.split(data, [3, 7, 11, 14])

    suhu = cekAja(splitKrom[0], splitData[0])
    waktu = cekAja(splitKrom[1], splitData[1])
    langit = cekAja(splitKrom[2], splitData[2])
    lembab = cekAja(splitKrom[3], splitData[3])
    
    hasil = np.all([suhu, waktu, langit, lembab])
    if (hasil) :
        return True
    else:
        return False
    
def panjangFinal(kromosom, data_train):
    kromosom = np.array(kromosom)
    rules = np.split(kromosom, len(kromosom)//15)
    
    for rule in rules:
        if(cekFinal(rule, data_train)):
            return True
    return False

In [16]:
for i in range(len(dt)):
    if panjangFinal(fitKrom[0][1],data_uji[i]):
        dt.loc[i, "Terbang/Tidak"] = "ya"
    else:
        dt.loc[i, "Terbang/Tidak"] = "tidak"
        
dt

,Suhu,Waktu,Kondisi Langit,Kelembapan,Terbang/Tidak
0,Tinggi,Siang,Berawan,Rendah,tidak
1,Rendah,Siang,Hujan,Tinggi,tidak
2,Normal,Pagi,Hujan,Tinggi,tidak
3,Tinggi,Siang,Hujan,Normal,tidak
4,Rendah,Malam,Hujan,Rendah,tidak
5,Normal,Sore,Cerah,Normal,tidak
6,Tinggi,Siang,Rintik,Rendah,tidak
7,Rendah,Siang,Cerah,Rendah,tidak
8,Normal,Siang,Cerah,Rendah,tidak
9,Tinggi,Siang,Hujan,Rendah,tidak


In [17]:
dt.to_csv("Tupro2.csv")